## Automate download of the zipped county files

In [2]:
# for the webpage (per state) automate the download of the ZIP county files as outlined below

In [ ]:
# Option 0
###
# parsing HTML code (downloaded from  USDA webapp pereodically - once a month? to check for updates there and download-unzip-ingest new versions of the files:
# this is the main URL where web app resides - it generates the table with county lists and timestamps when updated
# https://websoilsurvey.sc.egov.usda.gov/App/WebSoilSurvey.aspx 
# the source file download to /data_in/html/


In [123]:
import pandas as pd
import os 
from string import ascii_letters
import numpy as np

#Make a list from big HTML file
list = [ line for line in open(r'C:\Users\trubin\Desktop\Landviser LLC\git repos\soils-ssurgo\data_in\html\20220715_view-source_https___websoilsurvey.sc.egov.usda.gov_App_WebSoilSurvey.aspx.html') if '.zip' in line]
#List to dataframe
df = pd.DataFrame(list)
#name the column
df.columns = ['1']
#data cleaning
df['1'].str.split('                                                          ', expand=True)
df = df[df['1'].str.contains('TemplateDB') == False]
df['1'] = df['1'].map(lambda x: x.lstrip('</td></tr><tr><td class="line-number" value="'))
df[['1', '2']] = df['1'].str.split('                                                      <span class="html-tag">&lt;a <span class="html-attribute-name">href</span>="<a class="html-attribute-value html-external-link" target="_blank" href="', expand=True)
df = df.drop('1', axis=1)
df = df.replace(to_replace='None', value=np.nan).dropna()
df['1'] = df['2'].str.split('" rel=').str[0]
df['2'] = df['1'].str.split('" rel=').str[0]
df = df.drop('1', axis=1)
#result
df

,2
0,https://websoilsurvey.sc.egov.usda.gov/DSD/Dow...
2,https://websoilsurvey.sc.egov.usda.gov/DSD/Dow...
4,https://websoilsurvey.sc.egov.usda.gov/DSD/Dow...
6,https://websoilsurvey.sc.egov.usda.gov/DSD/Dow...
8,https://websoilsurvey.sc.egov.usda.gov/DSD/Dow...
10,https://websoilsurvey.sc.egov.usda.gov/DSD/Dow...
12,https://websoilsurvey.sc.egov.usda.gov/DSD/Dow...
14,https://websoilsurvey.sc.egov.usda.gov/DSD/Dow...
16,https://websoilsurvey.sc.egov.usda.gov/DSD/Dow...
18,https://websoilsurvey.sc.egov.usda.gov/DSD/Dow...


In [120]:
#export to csv
os.makedirs('C:/Users/trubin/Desktop/Landviser LLC/git repos/soils-ssurgo', exist_ok=True)
df.to_csv('C:/Users/trubin/Desktop/Landviser LLC/git repos/soils-ssurgo/data_out/links_list2.csv', index=False)  

Other options

In [ ]:
#Option 1
# Read zip files from page, download file, extract and stream output
from io import BytesIO
from zipfile import ZipFile
import urllib.request
import os,sys,requests,csv
from bs4 import BeautifulSoup

# check for download directory existence; create if not there
if not os.path.isdir('C:\\Users\\trubin\\Downloads'):
    os.makedirs('C:\\Users\\trubin\\Downloads')

# Get labels and zip file download links
mainurl = "https://websoilsurvey.sc.egov.usda.gov" # ??
#

# below is example of cached URL for zipfile with MS Access template included - we do not need it
# https://websoilsurvey.sc.egov.usda.gov/DSD/Download/Cache/SSA/wss_SSA_AL001_soildb_US_2003_[2021-09-15].zip

url = "http://websoilsurvey.sc.egov.usda.gov/DSD/Download/{CACHENAME}/{SUBFOLDER}/{FILENAME}"

# get page and setup BeautifulSoup
r = requests.get(url)
soup = BeautifulSoup(r.content, "html.parser")


# Get all file labels and filter so only use CSVs
mainlabel = soup.find_all("td", {"class": "labelOptional_ind"})
for td in mainlabel:
    if "_AL" in td.text:
        print(td.text)

###

# Get all <a href> urls
for link in soup.find_all('a'):
    print(mainurl + link.get('href'))

# QUESTION: HOW CAN I LOOP THROUGH ALL FILE LABELS AND FIND ONLY THE
# CSV LABELS AND THEIR CORRESPONDING ZIP DOWNLOAD LINK, SKIPPING ANY
# XML LABELS/LINKS, THEN LOOP AND EXECUTE THE CODE BELOW FOR EACH, 
# REPLACING zipfilename WITH THE MAIN LABEL AND zipurl WITH THE ZIP 
# DOWNLOAD LINK?

# Test downloading and streaming
zipfilename = "wss_SSA_AL019_[2021-09-15].zip"
zipurl = "http://websoilsurvey.sc.egov.usda.gov/DSD/Download/{CACHENAME}/{SUBFOLDER}/{FILENAME}"
outputFilename = "C:\\Users\\trubin\\Downloads" + zipfilename

# Unzip and stream CSV file
url = urllib.request.urlopen(zipurl)
zippedData = url.read()

# Save zip file to disk
print ("Saving to ",outputFilename)
output = open(outputFilename,'wb')
output.write(zippedData)
output.close()

# Unzip and stream CSV file
with ZipFile(BytesIO(zippedData)) as my_zip_file:
   for contained_file in my_zip_file.namelist():
    with open(("unzipped_and_read_" + contained_file + ".file"), "wb") as output:
        for line in my_zip_file.open(contained_file).readlines():
            print(line)


In [6]:
#Option 2
import urllib.request

user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'

url = "http://websoilsurvey.sc.egov.usda.gov"
headers={'User-Agent':user_agent,} 

request=urllib.request.Request(url,None,headers) #The assembled request
response = urllib.request.urlopen(request)
data = response.read() # The data u need

In [ ]:
#Option 3
opener = urllib.request.build_opener()
opener.addheaders = [('User-Agent', 'MyApp/1.0')]
urllib.request.install_opener(opener)
urllib.request.urlretrieve(
  "http://websoilsurvey.sc.egov.usda.gov/DSD/Download/{CACHENAME}/{SUBFOLDER}",
   "wss_SSA_AL019_[2021-09-15].zip")
outputFilename = "C:\\Users\\trubin\\Downloads" + zipfilename
url = urllib.request.urlopen(zipurl)
zippedData = url.read()
# Save zip file to disk
print ("Saving to ",outputFilename)
output = open(outputFilename,'wb')
output.write(zippedData)
output.close()

In [3]:
# create txt log of zipfiles downloaded


In [4]:
# do not download the file if already in the log

In [5]:
# if not in the log or the file on website is newer

# 1. download
# 2. update log
# 3. unzip to SSURGO folder
# 4. delete the downloaded zipped file